In [37]:
# Importing Packge
import chatterbot 
from chatterbot.trainers import ListTrainer
import os
import speech_recognition as sr
import pyttsx3
import json
import time
import requests

In [38]:
#create a bot
bot = chatterbot.ChatBot("NewBot")
# prepare trainer
trainer2 = ListTrainer(bot)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MHB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MHB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
# importing data sources and show the data files
folder = r"chatbotdata"
files = os.listdir(folder) 
files

['ai.txt', 'ai2.txt', 'ai3.txt', 'ai4.txt', 'ai5.txt']

In [40]:
# Bot training on all text file in destination folder 
for file in files:
    data = open(folder+"\\"+file).readlines()
    trainer2.train(data)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [41]:
# use Sapi5  engine for text to speech conversion
engine = pyttsx3.init("sapi5")
# reduce the rate of speech
engine.setProperty("rate", 150)
# changing the voice 
voices = engine.getProperty("voices")
voices
engine.setProperty("voice", voices[0].id)

In [42]:
# Telegram integration using Token
TOKEN = "567457:fhgjghfjjgdgh"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

In [43]:
#Creating functions 

def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    global js
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
    
# intilizing Telegram connection and sending test message
text, chat = get_last_chat_id_and_text(get_updates())
send_message("Test", chat)

# Telegram integration 

In [44]:
# connecting to Telegram
last_textchat = (None, None)
while True:
    text, chat = get_last_chat_id_and_text(get_updates())
    if (text, chat) != last_textchat:
        last_textchat = (text, chat)
        qus = text
        print("user say:",qus)
        ans=bot.get_response(qus)
        send_message(ans, chat)
        print("Bot: ",ans)

user say: Guhghhhhhg
Bot:  The activation function of a node defines the output of that node, or "neuron," given an input or set of inputs.
user say: Huff
Bot:  We can use linear regression for time series analysis, but the results are not promising. So, it is generally not advisable to do so. The reasons behind this are —
user say: Bshshs
Bot:  What is Deep Learning?


KeyboardInterrupt: 

# Testing the model through Typing

In [ ]:
# Typing the questions for testing 

userinput = input("What do you want to know about AI:  ")

res = bot.get_response(userinput)
#Displaying bot response
print("AI Says:", res)

# Listening to user and replying with text and speech

In [ ]:
# using google engine for speech to text conversion

# listening to user
r = sr.Recognizer()
with sr.Microphone(0) as source:
    audio = r.listen(source,phrase_time_limit=15)
text = r.recognize_google(audio)
#Displying user input
print("User Says:",text)
# bot response
res = bot.get_response(text)
#Displaying bot response
print("AI Says:", res)
# Use Sapi5 engine to convert bot response to speech
engine.say(res)
engine.runAndWait()

In [48]:
# gathering user Telegram first and last name

list=[]
for i in js["result"]:
    full_name2=0
    l=len(js["result"])
    for j in range(l):
        first_name=js["result"][j]["message"]["from"]["first_name"]
        last_name=js["result"][j]["message"]["from"]["last_name"]
        full_name=first_name+last_name
        if full_name not in list:
            list.append(full_name)
            full_name2=full_name
print(list)

['MohammedHamid Bawazir', 'Hamid Bawazir', 'Mohammed MarwanShahin', 'Shahin']


In [49]:
# saving user names
f=open("testfile.txt","a")
for i in list:
    f.write(i)
    f.write("\n")

In [50]:
# close the text file
f.close()

In [ ]:
# Dropping the database created in the session - to stop reinforcement learning
bot.storage.drop()